# Platform information

In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

# Parallel computation

In [2]:
library(doParallel)
library(foreach)
cl<-makeCluster(8) 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


# Loading data

In [3]:
geno <-read.table("./Hetero_realigned_cov10_filtered3.raw", row.names=1, header=T)
row.names(geno)=1:240
x <- as.matrix(geno)-1   
pheno <- read.csv("./2017heteroPheno.csv", header=T)
attach(pheno)
data <- data.frame(bcw=pheno$bcw,length=pheno$length,gid=1:240)

# Parameters for cross validation

In [4]:
repeats <- 10
n.fold <- 5 
n.sample <- length(pheno$bcw)    
CM<-7

# GBLUP

In [10]:
library(rrBLUP)
packageVersion("rrBLUP")

[1] '4.6'

In [11]:
#relationship matrix
A <- A.mat(x, n.core=8)
row.names(A)=1:240;colnames(A)=1:240

In [12]:
registerDoParallel(cl)
system.time({
GBLUP<-foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="bcw")
         cor(pheno$bcw[id==i],res$pred[id==i])
    }
   }
})
stopImplicitCluster()

   user  system elapsed 
   0.25    0.09    0.83 

# Bayes cpi

I found the code from this lab
 http://www.zzlab.net/StaGen/

In [5]:
source("http://www.zzlab.net/sandbox/BAGS.R")

In [8]:
registerDoParallel(cl)
system.time({
Cpi<-foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold) %dopar% {
         bcw_test <- data
         m1<-bcw_test$bcw[id != i] 
         u1<-as.matrix(geno)[id != i, ]        
         ki<-BAGS(u1, y=m1,pi=1,burn.in=1000,burn.out=2000)
         pred<-(as.matrix(geno)[id == i, ]) %*% ki$effect
         cor(pheno$bcw[id==i],pred)
    }
   }
})
stopImplicitCluster()

    user   system  elapsed 
    1.36     2.53 14754.27 

In [18]:
Cpi

result.1,0.2219468,0.1594255,0.1886464,0.3257046,0.008631627
result.2,0.04603167,0.329667,0.1702699,0.05681837,0.2823873
result.3,0.1618135,0.3293952,-0.08750843,0.1775165,-0.01163318
result.4,0.3945332,0.4198629,0.02603833,0.01668889,0.1229961
result.5,0.3297482,0.1469074,0.1622235,0.07863016,0.3126024
result.6,0.3477457,0.3905192,0.0518323,0.07439348,0.4505462
result.7,0.3303076,0.3233216,0.1655145,0.1465404,0.4664679
result.8,0.3482255,0.1964356,0.07313449,0.2668536,0.2184256
result.9,0.1701746,0.05899124,0.2461707,-0.09392551,0.2994342
result.10,0.1620705,0.1675907,0.1824422,0.2914718,0.1193513


# Bayes A, B, C, RR, LASSO, RKHS

In [11]:
library("BGLR")
packageVersion("BGLR")

[1] '1.0.5'

In [ ]:
registerDoParallel(cl)
system.time({
BA <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmBA=BGLR(y=bcw_test$bcw,ETA=list(list(X=x,model='BayesA')),nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmBA$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

In [40]:
registerDoParallel(cl)
system.time({
BB <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmBB=BGLR(y=bcw_test$bcw,ETA=list(list(X=x,model='BayesB')),nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmBB$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

   user  system elapsed 
   1.00    1.15   74.03 

In [42]:
registerDoParallel(cl)
system.time({
BC <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmBC=BGLR(y=bcw_test$bcw,ETA=list(list(X=x,model='BayesC')),nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmBC$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

   user  system elapsed 
   0.86    1.27   67.98 

In [43]:
registerDoParallel(cl)
system.time({
BRR <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmBRR=BGLR(y=bcw_test$bcw,ETA=list(list(X=x,model='BRR')),nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmBRR$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

   user  system elapsed 
   0.72    1.25   63.03 

In [44]:
registerDoParallel(cl)
system.time({
BL <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmBL=BGLR(y=bcw_test$bcw,ETA=list(list(X=x,model='BL')),nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmBL$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

   user  system elapsed 
   0.91    1.25   91.75 

In [46]:
registerDoParallel(cl)
system.time({
    p<-ncol(x)
    D<-(as.matrix(dist(x,method='euclidean'))^2)/p
    h<-0.5
    K<-exp(-h*D)
    ETA<-list(list(K=K,model='RKHS'))
RKHS <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %dopar% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmRKHS=BGLR(y=bcw_test$bcw,ETA=ETA,nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmRKHS$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

   user  system elapsed 
   1.40    0.09   18.25 

In [147]:
#Save result
Acc<-data.frame(unlist(GBLUP),unlist(BA),unlist(BB),unlist(BC),unlist(BRR),unlist(BL),unlist(RKHS))
colnames(Acc)<-c("GBLUP","BA","BB","BC","BRR","BL","RKHS")
library("xlsx")
write.xlsx(Acc, "7models_Acc.xlsx") 

# Comprision 

In [21]:
#Load data(from this page and deep learning models)
library("readxl")
data<-read_excel("9_models_comprision.xlsx")
Acc_all<-subset(data,select = - c(X__1))

In [23]:
Acc_all$BCpi<-unlist(Cpi)

In [24]:
(Acc_mean<-sapply(Acc_all,function(x) round(mean(x),digits = 3)))

GBLUP    BA    BB    BC   BRR    BL  RKHS    NN   MNN  BCpi 
0.286 0.282 0.296 0.285 0.283 0.284 0.283 0.300 0.307 0.196

In [25]:
(Acc_SE<-sapply(Acc_all,function(x) round(sd(x)/sqrt(repeats*n.fold),digits = 3)))

GBLUP    BA    BB    BC   BRR    BL  RKHS    NN   MNN  BCpi 
0.017 0.016 0.017 0.016 0.018 0.017 0.018 0.015 0.016 0.019